# RLDT


## Step 1: Import the necessary libraries:


In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

## Step 2: Define the environment:


#### ALL THE DEVICES


In [2]:
import ast

devices = pd.read_csv("devices.csv")
devices["voltages_frequencies"] = devices["voltages_frequencies"].apply(lambda x: ast.literal_eval(x))
devices["capacitance"] = devices["capacitance"].apply(
    lambda x: ast.literal_eval(x)
)
devices["occupied_cores"] = devices["occupied_cores"].apply(
    lambda x: ast.literal_eval(x)
)
devices["powerIdle"] = devices["powerIdle"].apply(
    lambda x: ast.literal_eval(x)
)
devices["acceptableTasks"] = devices["acceptableTasks"].apply(
    lambda x: ast.literal_eval(x)
)
devices = devices.drop(["Unnamed: 0"],axis=1)
# devices

,id,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,acceptableTasks,handleSafeTask
0,iot 0,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(40000000.0, 2.7), (80000000.0, 4.0), (20000...",0.18,"[2.8617377586262853e-10, 2.4976420199150807e-1...","[0.0009, 0.0009, 0.001, 0.0007999999999999999,...",3.900000e+10,0.01,"[3, 4, 2, 1]",1
1,iot 1,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(160000000.0, 5.0), (80000000.0, 4.0), (4000...",0.15,"[2.641855542991949e-10, 2.3092791861260476e-10...","[0.0007999999999999999, 0.001, 0.0007999999999...",3.600000e+10,0.04,"[1, 3, 4, 2]",1
2,iot 2,4,"[0, 0, 0, 0]","[[(10000000.0, 1.8), (160000000.0, 5.0), (4000...",0.11,"[2.120891569412965e-10, 2.4273370575229684e-10...","[0.0007999999999999999, 0.001, 0.0007999999999...",3.700000e+10,0.02,"[3, 2, 1]",1
3,iot 3,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(40000000.0, 2.7), (20000000.0, 2.3), (16000...",0.17,"[2.7596219485736067e-10, 2.1222521621518662e-1...","[0.0009, 0.0009, 0.001, 0.0009, 0.000799999999...",4.000000e+10,0.04,"[3, 2, 1]",1
4,iot 4,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(20000000.0, 2.3), (40000000.0, 2.7), (80000...",0.17,"[2.041383198645341e-10, 2.36568119849062e-10, ...","[0.0007999999999999999, 0.0009, 0.0009, 0.001,...",3.600000e+10,0.01,"[4, 3, 1]",1
5,iot 5,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(80000000.0, 4.0), (40000000.0, 2.7), (10000...",0.11,"[2.4292257212691467e-10, 2.003073007804602e-10...","[0.001, 0.0007999999999999999, 0.0009, 0.0009,...",3.800000e+10,0.05,"[4, 1, 3, 2]",1
6,iot 6,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(20000000.0, 2.3), (10000000.0, 1.8), (80000...",0.10,"[2.606954561068964e-10, 2.561245623196915e-10,...","[0.001, 0.001, 0.0009, 0.0009, 0.0009, 0.001, ...",3.600000e+10,0.03,"[3, 1, 4]",1
7,iot 7,8,"[0, 0, 0, 0, 0, 0, 0, 0]","[[(10000000.0, 1.8), (20000000.0, 2.3), (16000...",0.12,"[2.5851825057517555e-10, 2.509855688709864e-10...","[0.0009, 0.001, 0.0007999999999999999, 0.0009,...",4.000000e+10,0.04,"[1, 4, 2, 3]",1
8,iot 8,4,"[0, 0, 0, 0]","[[(160000000.0, 5.0), (40000000.0, 2.7), (2000...",0.10,"[2.7238478648415373e-10, 2.7861332896115326e-1...","[0.0009, 0.001, 0.001, 0.0007999999999999999]",3.800000e+10,0.05,"[3, 2, 1, 4]",1
9,iot 9,4,"[0, 0, 0, 0]","[[(20000000.0, 2.3), (10000000.0, 1.8), (16000...",0.16,"[2.225864956095805e-10, 2.9136305746601594e-10...","[0.001, 0.0007999999999999999, 0.0009, 0.00079...",3.600000e+10,0.04,"[2, 4, 3, 1]",1


### Step 2.2: Application


#### _Generate task DAGs_


In [3]:
tasks = pd.read_csv("tasks.csv")
tasks = tasks.drop(["Unnamed: 0"],axis=1)
tasks = tasks.set_index("id")
tasks

,id,job,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
0,t84462,6996,['t84460'],9,3,0,1969994,6992167,10449571,READY
1,t98682,8183,[],3,4,0,8067481,1707995,9393362,READY
2,t88344,7321,[],6,1,0,7291771,9229654,5588221,READY
3,t4297,365,['t4289'],4,2,0,2436514,9494920,2979157,READY
4,t62824,5205,"['t62815', 't62817', 't62820', 't62822']",5,4,0,3311581,9829107,7559947,READY
...,...,...,...,...,...,...,...,...,...,...
120330,t34743,2886,"['t34741', 't34742']",4,4,0,3071551,3006869,10925348,READY
120331,t69805,5768,"['t69799', 't69800']",4,3,0,4496246,1337876,9580426,READY
120332,t43199,3588,"['t43188', 't43189', 't43193', 't43197']",7,1,0,8313682,7384358,10085250,READY
120333,t90466,7499,"['t90464', 't90465']",7,3,0,7877120,9278025,1260630,READY


## Step 4 : DDT


### Step 4.1: Initializing The tree


In [4]:
class DDT(nn.Module):
    def __init__(self, num_input, num_output, depth, max_depth):
        super(DDT, self).__init__()
        self.depth = depth
        self.max_depth = max_depth
        if depth != max_depth:
            # self.weights = nn.Parameter(torch.zeros(num_input))
            self.weights = nn.Parameter(torch.empty(
                num_input).normal_(mean=0, std=0.1))
            self.bias = nn.Parameter(torch.zeros(1))
        if depth == max_depth:
            self.prob_dist = nn.Parameter(torch.zeros(num_output))

        if depth < max_depth:
            self.left = DDT(num_input, num_output, depth + 1, max_depth)
            self.right = DDT(num_input, num_output, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_dist
        val = torch.sigmoid(torch.matmul(x, self.weights.t()) + self.bias)
        a = np.random.uniform(0, 1)
        if a < 0.1:
            val = 1 - val
        if val >= 0.5:

            return val * self.right(x)
        else:

            return (1 - val) * self.left(x)

## Step 5: RL


In [5]:
def calc_execution_time(device, task, core, dvfs):
    if device['id'] == "cloud":
        return task["computationalLoad"] / device["voltages_frequencies"][0]
    else:
        return task["computationalLoad"] / device["voltages_frequencies"][core][dvfs][0]


def calc_power_consumption(device, task, core, dvfs):
    if device['id'] == "cloud":
        return 13.85 * calc_execution_time(device, task, core, dvfs)
    return (
        device["capacitance"][core]
        * (device["voltages_frequencies"][core][dvfs][1] ** 2)
        * device["voltages_frequencies"][core][dvfs][0]
    )
def calc_energy(device, task, core, dvfs):
    return calc_execution_time(device, task, core, dvfs) * calc_power_consumption(device, task, core, dvfs)


def calc_total(device, task, core, dvfs):
    timeTransMec = 0
    timeTransCC = 0
    exeTime = 0
    e = 0

    transferRate5g =1e9
    latency5g=5e-3
    transferRateFiber =1e10
    latencyFiber=1e-3

    timeDownMec = task["returnDataSize"] / transferRate5g
    timeDownMec += latency5g
    timeUpMec = task["dataEntrySize"] / transferRate5g
    timeUpMec += latency5g

    alpha = 52e-5
    beta = 3.86412
    powerMec = alpha * 1e9 / 1e6 + beta

    timeDownCC = task["returnDataSize"] / transferRateFiber
    timeDownCC += latencyFiber
    timeUpCC = task["dataEntrySize"] / transferRateFiber
    timeUpCC += latencyFiber

    powerCC = 3.65 


    if device["id"].startswith("mec"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec *  timeTransMec
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime + timeTransMec 
        e = calc_energy(device, task, core, dvfs)
        totalEnergy =  e + energyTransMec

    elif device['id'].startswith("cloud"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec * timeTransMec
        
        timeTransCC = timeUpCC+timeDownCC
        energyTransCC =  powerCC * timeTransCC
        
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime =  exeTime + timeTransMec +timeTransCC

        e = calc_energy(device, task, core, dvfs)
        totalEnergy =  + energyTransMec + energyTransCC

    elif device['id'].startswith("iot"):
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime
        e = calc_energy(device, task, core, dvfs)
        totalEnergy = e

    return totalTime , totalEnergy

In [6]:
def checkIfSuitable(state, device):
    punishment = 0
    safeFail = 0
    taskFail = 0
    if  state['safe'] and not device["handleSafeTask"]:
        punishment += 25
        safeFail += 1
        
    if state['kind'] not in device["acceptableTasks"]:
        punishment += 25
        taskFail += 1
    return (20 if punishment > 0 else 0, taskFail, safeFail)

In [7]:
def getSetup(t, e, setup, alpha=1, beta=1):
    match setup:
        case "00":
            reward = -1 * (e + t)
        case "01":
            reward = -1 * (alpha * e + beta * t)
        case "02":
            reward = -1 / (e + t)
        case "03":
            reward = -1 / (alpha * e + beta * t)
        case "04":
            reward = -np.exp(e) - np.exp(t)
        case "05":
            reward = -np.exp(alpha * e) - np.exp(beta * t)
        case "06":
            reward = -np.exp(t + e)
        case "07":
            reward = -np.exp(alpha * t + beta * e)
        case "08":
            reward = np.exp(-t - e)
        case "09":
            reward = np.exp(-1 * (alpha * t + beta * e))
    
    return reward

In [8]:
tasks_copy = tasks.copy()
tasks_copy = tasks_copy.drop(["job","dependency","mobility","status"],axis=1)
taskList = tasks_copy.index.tolist()

In [9]:
class Environment:
    def __init__(self):
        self.totalSafeFail = 0
        self.totalTaskFail = 0
        self.totalReward = 0
        self.feature_size = 5
        self.num_actions = len(devices)
        self.max_depth = 3
        self.agent = DDT(self.feature_size, self.num_actions,
                         depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)

    def execute_action(self, state, action):
        taskList.pop(0)
        device = devices.iloc[action]        

        punishment, taskFail, safeFail = checkIfSuitable(state, device)
        if safeFail:
            self.totalSafeFail += 1
        if taskFail:
            self.totalTaskFail += 1


        if not (punishment):
            for coreIndex in range(len(device["occupied_cores"])):
                if device["occupied_cores"][coreIndex] == 0:
                    total_t, total_e  = calc_total(device, state, coreIndex,np.random.randint(0,3))
                    reward = getSetup(total_t, total_e, "04")
                    # reward = -1 / (total_t + total_e)
                    # print(f"device {device['id']} ////// time {total_t}  ////// energy {total_e} ")
                    env.totalReward += reward
                    return (tasks_copy.loc[taskList[0]], reward, total_t, total_e)
        self.totalFail += 1
        return (tasks_copy.loc[taskList[0]], punishment, 0, 0)


    def train(self, num_epoch, num_episodes):

        total_avg_t = 0
        total_avg_e = 0
        total_avg_r = 0
        total_avg_l = 0

        for i in range(num_epoch):
            total_loss = 0
            env.totalFail = 0
            env.totalTaskFail = 0
            env.totalSafeFail = 0
            env.totalReward = 0
            total_loss = 0
            total_reward = 0
            totalTime = 0
            totalEnergy = 0


            
            for j in range(num_episodes):
                state = tasks_copy.loc[taskList[0]]
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)
                
                output = self.agent(x)
                action_probabilities = torch.softmax(output, dim=0)
                action_index = torch.multinomial(action_probabilities, 1).item()

                next_state, reward, t, e = self.execute_action(state, action_index)
                loss = (output[action_index] * reward)

                total_reward += reward
                total_loss += loss
                totalTime += t
                totalEnergy += e
                
                

            self.optimizer.zero_grad()
            avg_loss = total_loss/num_episodes
            avg_time = totalTime / num_episodes
            avg_energy = totalEnergy / num_episodes
            
            # avg_reward = total_reward / num_episodes
            avg_reward = env.totalReward / num_episodes
            avg_loss = total_loss/num_episodes


            total_avg_t += avg_time
            total_avg_e += avg_energy
            total_avg_l += avg_loss
            total_avg_r += avg_reward
            

            avg_loss.backward()
            self.optimizer.step()
            if i % 100 == 0:
                # print(f"Epoch {i+1} // avg cc time: {avg_cc} // avg mec: {avg_mec} // avg og time: {avg_og} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                # print(f"Epoch {i+1} // avg time: {avg_time} // avg added Time: {avg_added_time} // avg og time: {avg_og} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                print(f"Epoch {i+1}  // safe/task fail: {env.totalSafeFail}/{env.totalTaskFail} // Average Loss: {avg_loss:.2f} // Total Reward: {env.totalReward:.2f} // Average Reward: {avg_reward:.2f} // Avg time: {avg_time:.2f} // Avg energy: {avg_energy:.2f}")
            
            # if i == 10001:
                
            #     print(f"safe/task fail: {env.totalSafeFail}/{env.totalTaskFail} // Average Loss: {avg_loss:.2f} // Total Reward: {env.totalReward:.2f} // Average Reward: {avg_reward:.2f} // Avg time: {avg_time:.2f} // Avg energy: {avg_energy:.2f}")

                # env.totalFail = 0
                env.totalSafeFail = 0
                env.totalTaskFail

        avg_avg_t = total_avg_t / num_epoch
        avg_avg_l = total_avg_l / num_epoch
        avg_avg_r = total_avg_r / num_epoch
        avg_avg_e = total_avg_e / num_epoch


        print("===============================================================================")
        print(f'Overall Average Time across all epochs: {avg_avg_t}')
        print(f'Overall Average e across all epochs: {avg_avg_e:.2f}')
        print(f'Overall Average l across all epochs: {avg_avg_l:.2f}')
        print(f'Overall Average r across all epochs: {avg_avg_r:.2f}')
     





# env.totalAddedAvg += avg_cc


env = Environment()
tree = env.agent
env.train(1001, 10)

print('///////////////////')

for name, param in env.agent.named_parameters():
    if "prob_dist" or "bias" not in name:
        # print(name,param)
        pass

ValueError: could not convert string to float: 't84462'